# Inferencia aproximada y métodos de muestreo

<img style="float: right; margin: 0px 0px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/b/bf/Simple_random_sampling.PNG" width="400px" height="400px" />

> Hemos desarrollado métodos de **inferencia exacta** sobre redes Bayesianas. Aunque estos métodos explotan la estructura gráfica de nuestros modelos, no son computacionalmente tratables en el caso general, en el sentido de que el tiempo que necesitan para obtener un resultado es exponencial en el tamaño del problema.
>
> Por tanto, estudiaremos métodos que nos permitan realizar inferencia aproximada sobre redes Bayesianas, cuyo tiempo de cómputo puede ser significativamente menor (dependiendo de la precisión deseada).

> **Objetivos:**
> - Recordar cómo estimar valores esperados usando muestras.
> - Estudiar un método simple de muestreo para redes Bayesianas.

> **Referencias:**
> - Probabilistic Graphical Models: Principles and Techniques, By Daphne Koller and Nir Friedman. Ch. 12.
> - Mastering Probabilistic Graphical Models Using Python, By Ankur Ankan and Abinash Panda. Ch. 4.


<p style="text-align:right;"> Imagen recuperada de: https://upload.wikimedia.org/wikipedia/commons/b/bf/Simple_random_sampling.PNG.</p>

___

## 1. Estimación a partir de muestras

Antes que nada, recordemos cómo podemos estimar cosas sencillas usando muestras de una distribución $P$.

Sea $\mathcal{D}=\{x_1, x_2, \dots, x_N\}$ un conjunto de datos de muestras **independientes e identicamente distribuidas (IID)** de una distribución $P(X=x)$.

Por ejemplo, si $X$ es una VA binaria ($\mathrm{Val}(X)=\{x^0, x^1\}$) y $P(X=x^1)=p$, entonces un estimador para $p$es:

$$\hat{p} = \frac{1}{N} \sum_{i=1}^{N} I(x_i = x^1).$$

Veamos:

In [ ]:
# Import numpy


In [ ]:
# np.random.choice


In [ ]:
# Sample the binary distribution


In [ ]:
# Estimation of p


Más generalmente, para cualquier distribución $P$ y función $f$,

$$E_P[f] \approx \frac{1}{N} \sum_{i=1}^{N}f(x_i).$$

**Aplicación de este simple concepto:**

- [Montecarlo integration](https://en.wikipedia.org/wiki/Monte_Carlo_integration)

### ¿Cómo podemos muestrear de una distribución multinomial?

Sea $X$ una VA discreta, con $\mathrm{Val}(X)=\{x^1, x^2, \dots, x^k\}$, y $P(x^i) = \theta^i$.

Normalmente tendremos a la mano un generador de números pseudoaleatorio de la distribución uniforme $\mathcal{U}[0, 1]$ (probabilidad igual de obtener cualquier número entre 0 y 1):

In [ ]:
# numpy.random.rand


Podemos usar esto para muestrear la distribución multinomial como en la siguiente imagen:

In [1]:
from IPython.display import Image

In [ ]:
Image(filename='figures/discrete_sampling.png')

### ¿Qué garantías teóricas tenemos?

> *Teorema (cota de Hoeffding).* Para el estimador $\hat{p} = \frac{1}{N} \sum_{i=1}^{N} I(x_i = x^1)$, la desigualdad
> 
> $$P_{\mathcal{D}}(\hat{p}\notin[p-\epsilon, p+\epsilon]) \leq 2 e^{-2 N \epsilon^2}$$
>
> se satisface.

> *Teorema (cota de Chernoff).* Para el estimador $\hat{p} = \frac{1}{N} \sum_{i=1}^{N} I(x_i = x^1)$, la desigualdad
> 
> $$P_{\mathcal{D}}(\hat{p}\notin[p(1-\epsilon), p(1+\epsilon)]) \leq 2 e^{- N p \epsilon^2 / 3}$$
>
> se satisface.

**Comentarios:**

1. Hoeffding: error aditivo
2. Chernoff: errror multiplicativo
3. Ambos expresan que "la probabilidad de un dataset *malo* (con error $\epsilon$)" es una función decreciente del número de muestras.
4. Estas garantías se extienden a estimación de funciones generales.
5. Problemas cuando $p$ (el valor real) es pequeño: $\epsilon$ debe ser pequeño relativo a $p$.

**Ejercicio.**

Suponga que queremos estar $1-\delta$ seguros que tendremos un buen estimador. ¿Cuántas muestras necesitamos?

## 2. Muestreo hacia adelante de redes Bayesianas

Existen varias maneras de muestrear una red Bayesiana. El *muestreo hacia adelante* es quizá la manera más fácil y práctica de hacerlo, dado que se hace en la dirección causal del modelo.

Consideremos la red del estudiante:

In [ ]:
Image(filename='figures/Student1.png')

El muestreo se hace en dirección causal:

1. Comenzamos con los nodos que no tienen nodos padres ($D,I$):
   - $d^0, i^1$.

2. Una vez los nodos padres de un nodo han sido muestrados, podemos muestrear dicho nodo usando la fila correspondiente de la distribución condicional ($C,E$ usando las filas correspondientes a $i^1, d^0$):
   - $c^0, e^0$.
   - $r^1$

3. Finalmente, la muestra completa es $d^0, i^1, c^0, e^0, r^1$.

### ¿Cómo estimamos probabilidades usando estas muestras?

1. Objetivo: estimar $P(\bar{Y}=\bar{y})$ (o una función de $\bar{Y}$).

   - Generar muestras de la red Bayesiana. ¿Cuántas?
   - Contar el número de veces que $\bar{Y}=\bar{y}$ y calcular su frecuencia relativa.

### ¿Y si tenemos evidencia?

2. Objetivo: estimar $P(\bar{Y}=\bar{y} | \bar{E}=\bar{e})$.

   - Algoritmo de muestreo y rechazo:
     - Generar muestras de la red Bayesiana.
     - Tirar las muestras en las que $\bar{E} \neq \bar{e}$
     - Contar las veces que $\bar{Y}=\bar{y}$ y calcular su frecuencia relativa.
     
   - Fracción esperada de muestras que quedan después de tirar las muestras no consistentes con la evidencia: $P(\bar{e})$.
   - Por tanto, deberíamos generar: $N\geq \frac{\log(2/\delta)}{2P(\bar{e})\epsilon^2}$ samples

> El número de muestras crece con el número de variables observadas.

### ¿Cómo se hace con `pgmpy`?

Inferencia exacta:

In [ ]:
# Importamos pgmpy.models.BayesianModel
from pgmpy.models import BayesianModel
# Importamos pgmpy.factors.discrete.TabularCPD
from pgmpy.factors.discrete import TabularCPD

In [ ]:
# Definimos el esqueleto de la red mediante los arcos


In [ ]:
# Definimos distribución condicional de D
cpd_D = TabularCPD(variable='D',
                   variable_card=2,
                   values=[[0.6],
                           [0.4]])
# Definimos distribución condicional de I
cpd_I = TabularCPD(variable='I',
                   variable_card=2,
                   values=[[0.7],
                           [0.3]])
# Definimos distribución condicional de C
cpd_C = TabularCPD(variable='C',
                   variable_card=3,
                   evidence=['I', 'D'],
                   evidence_card=[2, 2],
                   values=[[0.30, 0.70, 0.02, 0.20],
                           [0.40, 0.25, 0.08, 0.30],
                           [0.30, 0.05, 0.90, 0.50]])
# Definimos distribución condicional de P
cpd_E = TabularCPD(variable='E',
                   variable_card=2,
                   evidence=['I'],
                   evidence_card=[2],
                   values=[[0.95, 0.2],
                           [0.05, 0.8]])
# Definimos distribución condicional de R
cpd_R = TabularCPD(variable='R',
                   variable_card=2,
                   evidence=['C'],
                   evidence_card=[3],
                   values=[[0.99, 0.4, 0.1],
                           [0.01, 0.6, 0.9]])

In [ ]:
# Associating the CPDs with the network
student_model.add_cpds(cpd_D, cpd_I, cpd_C, cpd_E, cpd_R)

In [ ]:
student_model.check_model()

In [ ]:
# Import pgmpy.inference.VariableElimination


Muestreo hacia adelante:

In [ ]:
# Import pgmpy.sampling.BayesianModelSampling


In [ ]:
# Instantiate a sampling object


In [ ]:
# Number of samples for error of 1% and confidence of 99%


In [ ]:
# Generate samples


In [ ]:
# Estimation of P(C)


In [ ]:
# Compare


<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>